In [1]:
import pandas as pd
import numpy as np
import time
from pulp import *

In [2]:
distMat = pd.read_csv('Dist.csv',index_col='idx')
processingTech = pd.read_csv('techs.csv',index_col='idx')
extractionLevel = pd.read_csv('levels.csv',index_col='idx')

In [3]:
distMat

,G1,G2,G3,G4,G5,G6,G7,G8,E1,E2
idx,,,,,,,,,,
G1,0,200,450,450,200,400,150,250,300,600
G2,200,0,300,300,300,150,300,300,150,600
G3,450,300,0,50,300,100,450,200,200,700
G4,450,300,50,0,200,200,400,150,180,500
G5,200,300,300,200,0,400,200,100,300,500
G6,400,150,100,200,400,0,450,350,200,900
G7,150,300,450,400,200,450,0,300,400,800
G8,250,300,200,150,100,350,300,0,300,600
E1,999,999,999,999,999,999,999,999,0,600


In [4]:
processingTech

,capacity,fixedCost,runningCost,cruidOil,seedCake,oilInSeedCake
idx,,,,,,
T1,6000,875000,500,250,750,7.5
T2,6000,500000,380,150,850,12.5
T3,12000,625000,320,166,834,16.0


In [5]:
extractionLevel

,capacity,fixedCost,runningCost
idx,,,
low,9000,625000,445
med,30000,1500000,415
high,150000,6000000,400


In [6]:
growingAreas = list(distMat.index)[:-2]
technologies = list(processingTech.index)
levels = list(extractionLevel.index)
extractionP = ['E1','E2']
plant = ['E2']
print(growingAreas)
print(technologies)
print(levels)
print(extractionP)

['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8']
['T1', 'T2', 'T3']
['low', 'med', 'high']
['E1', 'E2']


In [7]:
processingTech.loc['T1','fixedCost']

875000

In [8]:
seed = LpVariable.dicts('seed',(growingAreas,growingAreas,technologies),lowBound=0,upBound=1500)
plantInd = LpVariable.dicts('z',growingAreas,cat='Binary')
plantTechInd = LpVariable.dicts('g',(growingAreas,technologies),cat='Binary')
extractionInd = LpVariable.dicts('e',extractionP,cat='Binary')
extractionLevelInd = LpVariable.dicts('e',(extractionP,levels),cat='Binary')
cake = LpVariable.dicts('cake',(growingAreas,technologies,extractionP,levels),lowBound=0)
oilPlant = LpVariable.dicts('oil1',(growingAreas,plant),lowBound=0)
oilExtraction = LpVariable.dicts('oil2',(extractionP,plant),lowBound=0)
totalOil = LpVariable('to',lowBound=0)
cost = LpVariable('cost',lowBound=0)

In [9]:
scm = LpProblem('SCM',LpMaximize)

In [10]:
#Objective
scm += lpSum(lpSum(seed[ga][ga1][t] for t in technologies)*distMat.loc[ga,ga1] for 
             ga in growingAreas for ga1 in growingAreas)\
+ lpSum(lpSum(plantTechInd[ga][t] for ga in growingAreas)*processingTech.loc[t,'fixedCost'] for t in technologies)\
+ lpSum(lpSum(seed[ga][ga1][t] for ga in growingAreas for ga1 in growingAreas)
        *processingTech.loc[t,'runningCost'] for t in technologies)\
+ lpSum(oilPlant[ga][pl]*distMat.loc[ga,pl] for ga in growingAreas for pl in plant)\
+ lpSum(lpSum(cake[ga][t][e][l] for t in technologies for l in levels)*distMat.loc[ga,e] for 
        ga in growingAreas for e in extractionP)\
+ lpSum(lpSum(extractionLevelInd[e][l] for e in extractionP)*extractionLevel.loc[l,'fixedCost'] for l in levels)\
+ lpSum(lpSum(cake[ga][t][e][l] for ga in growingAreas for t in technologies for e in 
              extractionP)*extractionLevel.loc[l,'runningCost'] for l in levels)\
+lpSum(oilExtraction[e][p]*distMat.loc[e,p] for e in extractionP for p in plant)

In [11]:
for ga1 in growingAreas:
    scm += lpSum(seed[ga][ga1][t] for ga in growingAreas for t in technologies) <= 1503*8*plantInd[ga1]
    scm += lpSum(plantTechInd[ga1][t] for t in technologies) == plantInd[ga1]

In [12]:
for ga in growingAreas:
    scm += lpSum(seed[ga][ga1][t] for ga1 in growingAreas for t in technologies) == 1500

In [13]:
for ga1 in growingAreas:
    lpSum(seed[ga][ga1][t] for ga in growingAreas for t in technologies) <= lpSum(
        plantTechInd[ga1][t]*processingTech.loc[t,'capacity'] for t in technologies)

In [14]:
for ga1 in growingAreas:
    for ref in plant:
        scm += oilPlant[ga1][ref] == lpSum(lpSum(seed[ga][ga1][t]*processingTech.loc[t,'cruidOil']/1000 for 
                                           t in technologies) for ga in growingAreas)

In [15]:
for ga1 in growingAreas:
    for t in technologies:
        scm += lpSum(cake[ga1][t][e][l] for e in extractionP for l in levels) == lpSum(
            lpSum(seed[ga][ga1][t]*processingTech.loc[t,'seedCake']/1000) for ga in growingAreas)

In [16]:
for e in extractionP:
    for ref in plant:
        scm += oilExtraction[e][ref] == lpSum(cake[ga1][t][e][l]*processingTech.loc[t,'oilInSeedCake']/100 for ga1 
                                              in growingAreas for t in technologies for l in levels)

In [17]:
for e in extractionP:
    for l in levels:
        scm += lpSum(cake[ga1][t][e][l] for ga1 in growingAreas for t in technologies) <= \
        extractionLevelInd[e][l]*extractionLevel.loc[l,'capacity']

In [18]:
for e in extractionP:
    scm += extractionInd[e] == lpSum(extractionLevelInd[e][l] for l in levels)

In [19]:
for e in extractionP:
    scm += lpSum(cake[ga1][t][e][l] for ga1 in growingAreas for t in technologies for l in levels) <= 1503*8*extractionInd[e]

In [20]:
for ga1 in growingAreas:
    for t in technologies:
        scm += lpSum(cake[ga1][t][e][l] for e in extractionP for l in levels) <= 1503*8*plantTechInd[ga1][t]

In [21]:
for ga1 in growingAreas:
    scm += lpSum(oilPlant[ga1][ref] for ref in plant) <= 1503*8*plantInd[ga1]

In [22]:
#scm += extractionInd['E2']==1
#scm += extractionInd['E1']==0
#scm += totalOil == lpSum(oilPlant[ga1]['E2'] for ga1 in growingAreas) + lpSum(oilExtraction[e]['E2'] for e in extractionP)
#scm += totalOil >= 3675

In [23]:
scm.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: May 10 2021 

command line - cbc /var/folders/sp/0pfsflvj2yb1cnz2gjryrtnh0000gn/T/d30dc907da504c10a81f473babaa9cc9-pulp.mps max branch printingOptions all solution /var/folders/sp/0pfsflvj2yb1cnz2gjryrtnh0000gn/T/d30dc907da504c10a81f473babaa9cc9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 105 COLUMNS
At line 2155 RHS
At line 2256 BOUNDS
At line 2489 ENDATA
Problem MODEL has 100 rows, 386 columns and 1594 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 4.36e+07 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 16 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 16 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 16 strengthened rows, 0 substitutions
Cgl0004I processed model has 90 rows, 376 columns (40 integer (40 of which binary)) and 1480 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 1.66533e-1

1

In [24]:
for ga in growingAreas:
    if plantInd[ga].varValue !=0:
        print(f"Processing plant is opened at {ga} growing area")

Processing plant is opened at G1 growing area
Processing plant is opened at G2 growing area
Processing plant is opened at G3 growing area
Processing plant is opened at G4 growing area
Processing plant is opened at G5 growing area
Processing plant is opened at G6 growing area
Processing plant is opened at G7 growing area
Processing plant is opened at G8 growing area


In [25]:
for ga in growingAreas:
    for t in technologies:
        if plantTechInd[ga][t].varValue !=0:
            print(f'Processing plant is opened at growing area {ga}, working at technology {t}')

Processing plant is opened at growing area G1, working at technology T1
Processing plant is opened at growing area G2, working at technology T1
Processing plant is opened at growing area G3, working at technology T1
Processing plant is opened at growing area G4, working at technology T1
Processing plant is opened at growing area G5, working at technology T1
Processing plant is opened at growing area G6, working at technology T1
Processing plant is opened at growing area G7, working at technology T1
Processing plant is opened at growing area G8, working at technology T1


In [26]:
for ga in growingAreas:
    for ga1 in growingAreas:
        for t in technologies:
            if seed[ga][ga1][t].varValue !=0:
                print(f'{ga} Growing areas {seed[ga][ga1][t].varValue} yeild is going to PP at {ga1} at tech {t}')

G1 Growing areas 1500.0 yeild is going to PP at G6 at tech T1
G2 Growing areas 1500.0 yeild is going to PP at G7 at tech T1
G3 Growing areas 1500.0 yeild is going to PP at G7 at tech T1
G4 Growing areas 1500.0 yeild is going to PP at G7 at tech T1
G5 Growing areas 1500.0 yeild is going to PP at G6 at tech T1
G6 Growing areas 1500.0 yeild is going to PP at G7 at tech T1
G7 Growing areas 1500.0 yeild is going to PP at G6 at tech T1
G8 Growing areas 1500.0 yeild is going to PP at G6 at tech T1


In [27]:
for e in extractionP:
    for l in levels:
        if extractionLevelInd[e][l].varValue !=0:
            print(f'extraction plant {e} is opened at level {l}')

extraction plant E1 is opened at level high
extraction plant E2 is opened at level high


In [28]:
for ga1 in growingAreas:
    for t in technologies:
        for e in extractionP:
            for l in levels:
                if cake[ga1][t][e][l].varValue !=0:
                    print(f'Seed Cake {cake[ga1][t][e][l].varValue} is sent from PP {ga1} at tech {t} to EP {e} at level {l}')

Seed Cake 4500.0 is sent from PP G6 at tech T1 to EP E2 at level high
Seed Cake 4500.0 is sent from PP G7 at tech T1 to EP E2 at level high


In [29]:
scm.isMIP()

1

In [30]:
scm.unusedConstraintName()

'_C101'

In [31]:
total_oil = 0
for ga1 in growingAreas:
    for ref in plant:
        total_oil += oilPlant[ga1][ref].varValue
for e in extractionP:
    for ref in plant:
        total_oil += oilExtraction[e][ref].varValue
total_oil

3675.0